# Forecasting

In [ ]:
! pip install pandas openpyxl numpy scikit-learn prophet matplotlib seaborn jupyter ipywidgets

In [ ]:
%load_ext autoreload
%autoreload 2


In [157]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [158]:
from utils import forecasting

In [159]:
import os 

if not os.path.exists('output'):
    os.mkdir('output')

# Inital Data Extraction (Only run once)

In [9]:
# dfs = pd.read_excel("data/SES_2024.xlsx.coredownload.xlsx", sheet_name="T3.5",header=[3, 4])


In [ ]:
# df_region = dfs.iloc[:49, :]
# df_region

In [25]:
# region_index = df_region[('Overall', 'Unnamed: 0_level_1')].to_list()

In [53]:
# df = df_region.loc[:, df_region.columns.get_level_values(1).astype(str).str.isnumeric()]
# df.index = region_index

# df.columns = pd.to_datetime([f"{year}-{month}-01" for year, month in df.columns])
# df.to_csv('data/region_monthly_electricity_consumption.csv', index=True)


# Data Preparation for Forecasting 

In [170]:
df = pd.read_csv('data/region_monthly_electricity_consumption.csv', index_col=0)

df.head()

,2005-01-01,2005-02-01,2005-03-01,2005-04-01,2005-05-01,2005-06-01,2005-07-01,2005-08-01,2005-09-01,2005-10-01,...,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01
Overall,418.1,408.3,446.9,496.2,496.8,519.7,497.7,489.3,477.7,480.8,...,449.7,463,457.3,415.9,384.1,394.9,423.7,491,482.9,464.1
Central Region,485.4,459,500.2,566.4,573.3,590.1,568.3,551.7,539.5,557.6,...,505.4,523.1,527.6,481.4,442.7,436.7,478.9,553.5,558.8,524.8
Bishan,433.6,420.5,479.2,513.5,516.3,545,522.6,507.7,505.8,508,...,463,495.7,471.5,429.1,379.6,416.1,471.4,498,485.4,489.9
Bukit Merah,320,264.9,292.4,358.7,354.2,350,358.4,337.8,343.2,344.9,...,368.9,364.2,384.9,346.4,341.6,301.1,330.2,389.9,421.9,380.2
Bukit Timah,963.4,930.5,1035.2,1149.5,1132.3,1173.7,1101.2,1051.2,1039,1109.4,...,911.5,951,916.9,834.9,797.2,760.7,876.8,1012.1,938.6,856.5


In [171]:
df.index

Index(['Overall', 'Central Region', 'Bishan', 'Bukit Merah', 'Bukit Timah',
       'Downtown Core', 'Geylang', 'Kallang', 'Marine Parade', 'Museum',
       'Newton', 'Novena', 'Orchard', 'Outram', 'Queenstown', 'River Valley',
       'Rochor', 'Singapore River', 'Southern Islands', 'Tanglin', 'Toa Payoh',
       'East Region', 'Bedok', 'Changi', 'Pasir Ris', 'Paya Lebar', 'Tampines',
       'North East Region', 'Ang Mo Kio', 'Hougang', 'Punggol', 'Seletar',
       'Sengkang', 'Serangoon', 'North Region', 'Mandai', 'Sembawang',
       'Sungei Kadut', 'Woodlands', 'Yishun', 'West Region', 'Bukit Batok',
       'Bukit Panjang', 'Choa Chu Kang', 'Clementi', 'Jurong East',
       'Jurong West', 'Pioneer', 'Tengah'],
      dtype='object')

In [ ]:
df_long, df_stats = forecasting.prepare_electricity_data(df=df)
df_long

In [ ]:
seasonality_df = forecasting.analyze_seasonality(df_long=df_long)

peak_counts = seasonality_df['peak_month'].value_counts()
low_counts = seasonality_df['low_month'].value_counts()

months_df = pd.DataFrame({
    'Month': peak_counts.index.append(low_counts.index),
    'Type': ['Peak'] * len(peak_counts) + ['Low'] * len(low_counts),
    'Count': pd.concat([peak_counts, low_counts]).values
})

# Plotting the results
plt.figure(figsize=(10, 6))
sns.barplot(data=months_df, x='Month', y='Count', hue='Type', palette='pastel')
plt.title('Frequency of Peak and Low Months Across Regions')
plt.ylabel('Count')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.legend(title='Month Type')

plt.savefig('output/peak_low_months_frequency.png')
plt.show()

In [ ]:
anomalies = forecasting.identify_anomalies(df_long=df_long, z_score_threshold=3)
top_10_anomalous_regions = anomalies.groupby('region')['region'].value_counts().sort_values(ascending=False)[:10]
top_10_anomalous_regions

In [ ]:
forecasting.plot_anomalies(df_long=df_long, anomalies=anomalies, regions_to_plot=top_10_anomalous_regions.index, save=True)

Many anomalies during Covid period. High electricity consumption due to the lockdown. Tengah anomalies are due to the region being a new BTO estate.

In [ ]:
print(f"Earliest Date: {min(df_long['date'])}")
print(f"Latest Date: {max(df_long['date'])}")

In [ ]:
print("=== Top 10 Regions with Highest Electricity Consumption ===\n")
df_long.groupby(['region_category','region'])['consumption'].mean().sort_values(ascending=False)[:10]


In [ ]:
# Run the forecasting
results = forecasting.train_forecasting_model(df_long=df_long, region_category='Region')


# Print metrics
metrics_df = forecasting.print_metrics(results)
print("\nModel Performance Metrics:")
print(metrics_df)

# Plot forecasts for key regions
regions_to_plot = metrics_df['Region']
fig = forecasting.plot_forecasts(results=results, regions_to_plot=regions_to_plot, save=True)
plt.show()

# Get future predictions for Overall consumption
overall_forecast = results['Overall']['forecast'].tail(6)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
print("\nForecast for Overall consumption (next 6 months):")
print(overall_forecast.round(2))


In [ ]:
results['Overall']

In [ ]:
results['Overall']['forecast']